In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.nn_data_classifier import load_data, Classifier
from utils.preprocess import preprocess, RNNDataset
writer = SummaryWriter(log_dir='logs')



Load the dataset

In [2]:
historical_data = load_data()

# historical_data['minute'] = historical_data.Day.dt.minute
# historical_data = historical_data[historical_data.minute == 0]
# historical_data.drop('minute', axis=1, inplace=True)

historical_data

,Day,open,high,low,Price,Volume BTC,Volume
525598,2017-01-01 00:01:00,966.34,966.34,966.34,966.34,7.610000,7353.847400
525597,2017-01-01 00:02:00,966.16,966.37,966.16,966.37,8.087376,7815.397864
525596,2017-01-01 00:03:00,966.37,966.37,966.37,966.37,0.000000,0.000000
525595,2017-01-01 00:04:00,966.37,966.37,966.37,966.37,0.000000,0.000000
525594,2017-01-01 00:05:00,966.43,966.43,966.43,966.43,0.107000,103.408010
...,...,...,...,...,...,...,...
2064523,2022-03-01 03:38:00,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143
2064522,2022-03-01 03:39:00,43035.16,43035.16,42999.44,42999.44,0.820950,35300.390268
2064521,2022-03-01 03:40:00,43022.24,43022.24,43016.03,43016.03,0.009230,397.037957
2064520,2022-03-01 03:41:00,43018.23,43046.59,43018.23,43046.58,0.142977,6154.673021


Classify data

In [3]:
time_outlook = 3
classi = Classifier(historical_data)
classified_data = classi.classify_data_strict_time(time_outlook=time_outlook)
classified_data

,Day,open,high,low,Price,Volume BTC,Volume,Classification
525598,2017-01-01 00:01:00,966.34,966.34,966.34,966.34,7.610000,7353.847400,1
525597,2017-01-01 00:02:00,966.16,966.37,966.16,966.37,8.087376,7815.397864,1
525596,2017-01-01 00:03:00,966.37,966.37,966.37,966.37,0.000000,0.000000,1
525595,2017-01-01 00:04:00,966.37,966.37,966.37,966.37,0.000000,0.000000,1
525594,2017-01-01 00:05:00,966.43,966.43,966.43,966.43,0.107000,103.408010,1
...,...,...,...,...,...,...,...,...
2064526,2022-03-01 03:35:00,43098.34,43098.42,43083.17,43083.17,0.609030,26238.943025,0
2064525,2022-03-01 03:36:00,43095.57,43095.57,43068.85,43090.55,0.068882,2968.151200,0
2064524,2022-03-01 03:37:00,43078.73,43092.09,43078.73,43088.92,1.593095,68644.735251,0
2064523,2022-03-01 03:38:00,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143,0


Sort data

In [4]:
data = classified_data.sort_values('Day').reset_index(drop=True)
data

,Day,open,high,low,Price,Volume BTC,Volume,Classification
0,2017-01-01 00:01:00,966.34,966.34,966.34,966.34,7.610000,7353.847400,1
1,2017-01-01 00:02:00,966.16,966.37,966.16,966.37,8.087376,7815.397864,1
2,2017-01-01 00:03:00,966.37,966.37,966.37,966.37,0.000000,0.000000,1
3,2017-01-01 00:04:00,966.37,966.37,966.37,966.37,0.000000,0.000000,1
4,2017-01-01 00:05:00,966.43,966.43,966.43,966.43,0.107000,103.408010,1
...,...,...,...,...,...,...,...,...
2675293,2022-03-01 03:35:00,43098.34,43098.42,43083.17,43083.17,0.609030,26238.943025,0
2675294,2022-03-01 03:36:00,43095.57,43095.57,43068.85,43090.55,0.068882,2968.151200,0
2675295,2022-03-01 03:37:00,43078.73,43092.09,43078.73,43088.92,1.593095,68644.735251,0
2675296,2022-03-01 03:38:00,43077.82,43077.82,43049.46,43049.46,0.022210,956.143143,0


In [5]:
SEQ_LEN = 60

In [6]:
processor = preprocess(sequence_length=SEQ_LEN)
training, validation, testing  = processor.preprocess(dataframe=data, validation_size=0.2)

In [7]:
from torch import nn
class RNN_module(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, num_layers):
        super(RNN_module, self).__init__()
        self._num_layers = num_layers
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.lstm = nn.LSTM(input_size = self._input_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)
        self.fc = nn.Linear(in_features=self._hidden_size, out_features= self._output_size)

    def __str__(self):
        return f"RNN LSTM Model w/ {self._input_size} features and {self._num_layers} layers and {self._hidden_size} of hidden size"

    def forward(self, input):
        h_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)
        c_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)

        lstm_output, (h_n, c_n) = self.lstm(input)

        pred = self.fc(lstm_output[:, -1, :])
        
        return pred

In [8]:
from torch.utils.data import DataLoader

learning_rate = 0.01
dim_size = training._features.size(dim=-1)
hidden_size = 60
batch_size = 64
number_of_classes = training[1][1].shape[0]

train_dataloader = DataLoader(training, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(validation, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testing, batch_size = batch_size, shuffle = False)

model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

In [9]:
labels = training._labels
classes = torch.unique(labels, dim = 0)

freq = {}
for _class in classes:
    freq[_class.item()] = 0

weights = torch.zeros((classes.shape[0],))
total = torch.tensor(len(training))

for input in range(training._beg, training._end):
    _, output = torch.max(training[input][1], 0)
    freq[output.item()] += 1


for idx, _class in enumerate(classes):
    weights[idx] +=  1 - freq[_class.item()]/total

weights

In [ ]:
criterion = nn.CrossEntropyLoss(weight= weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
CHECKPOINT_PATH = './models_parameters/LSTM/checkpoints_2/'
BEST_PATH = './models_parameters/LSTM/best_model.pth'

def epoch_training(train_dataloader, epoch, total_epochs):
    running_correct = 0
    running_loss = 0.0
    n_of_steps = len(train_dataloader)

    for current_batch, (sequence, label) in enumerate(train_dataloader):
        #forward: we are calculating the loss given the parameters
        outputs = model(sequence)
        loss = criterion(input=outputs, target = label.float())

        #backward: lets update the parameters given the current loss
        optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
        loss.backward() #computates the bwrd-prop gradient for each model parameter
        optimizer.step() #updates the model current parameter using the gradients.

        predictions = torch.argmax(outputs, 1)
        correct = torch.argmax(label, 1)

        running_correct += (predictions == correct).sum().item()
        running_loss += loss.item()

        if (current_batch + 1) % 200 == 0:
            print(f"epoch {epoch+1}/{total_epochs}, current step(batch): {current_batch+1}/{n_of_steps}, loss = {loss.item():.4f} ")
            writer.add_scalar('training loss: ', running_loss/200, epoch * n_of_steps + current_batch)
            writer.add_scalar('accuracy: ', running_correct/200, epoch * n_of_steps + current_batch)
            running_loss = 0.0
            running_correct = 0

    writer.add_scalar('Epoch loss: ', loss, epoch + 1)


def epoch_validate(validation_dataloader, epoch, total_epochs):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0
        
        for current_batch, (sequence, label) in enumerate(validation_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)
            
            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

        acc = 100.0 * n_corrects / n_samples

        print(f"epoch {epoch+1}/{total_epochs} accuracy: {acc}")
        writer.add_scalar('Validation Accuracy: ', acc, epoch+1)

    return acc


def train_loop(train_dataloader: DataLoader, validation_dataloader: DataLoader, epochs: int):
    
    max_accuracy = 0
    is_best = False

    for epoch in range(epochs):
        epoch_training(train_dataloader, epoch, epochs)

        accuracy = epoch_validate(validation_dataloader, epoch, epochs)

        if accuracy > max_accuracy:
            is_best = True
            max_accuracy = accuracy
        else:
            is_best = False
        
        checkpoint = {
            'epoch': epoch+1,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict()
        }

        if is_best:
            torch.save(checkpoint, BEST_PATH)
        
        torch.save(checkpoint, CHECKPOINT_PATH+f'model_{epoch+1}.pth')

In [ ]:
def test_loop(test_dataloader: DataLoader, model: nn.Module):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(test_dataloader):
            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)

            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

            if (current_batch + 1) % 200 == 0:
                print(f"test batch: {current_batch+1}/{len(test_dataloader)}, current accuracy: {100 * n_corrects / n_samples}")

        acc = 100.0 * n_corrects / n_samples
        print(f"final test accuracy: {acc}")


In [ ]:
epochs = 10
train_loop(train_dataloader=train_dataloader, validation_dataloader = validation_dataloader, epochs=epochs)
test_loop(test_dataloader=test_dataloader, model=model) #curioso: quando o dataloader tava em lista (tuple) o modelo tava treinando bem mais rápido...

Using best model in validation

In [ ]:
best_model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

checkpoint = torch.load(BEST_PATH)
print(f'Model type: {best_model}')
print(f'Best performing model found at {checkpoint["epoch"]}ºepoch')

best_model.load_state_dict(state_dict=checkpoint['model_state'], strict=True)
best_model.eval()

test_loop(test_dataloader=test_dataloader, model=best_model)

In [ ]:
writer.close()